In [6]:
import gymnasium as gym
import collections
import numpy as np
import statistics
import tensorflow as tf
import tensorflow_probability as tfp
import tqdm
import os

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple
from tensorflow.python.framework.ops import get_gradient_function

In [7]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
class ContinuousActor(tf.keras.Model):
    """Actor that outputs a policy directly"""
    def __init__(
        self,
        num_actions: int,
        num_hidden_units: int,
        std_init = -2
    ):
        super().__init__()
        self.means = layers.Dense(num_actions)
        self.stds = tf.Variable(tf.zeros(num_actions))#*std_init#lambda x: tf.zeros(num_actions)
        # self.stds = layers.Dense(num_actions, activation='relu')
        # tf.ones(num_actions) * 0.25
    def call(self, inputs: tf.Tensor):
        means = self.means(inputs)
        # stds = self.stds(inputs)
        # stds = tf.clip_by_value(stds, 1.0e-3, 1)
        return tfp.distributions.MultivariateNormalDiag(loc = means, scale_diag = tf.exp(self.stds))


class ActorCritic(tf.keras.Model):
    """combined actor-critic network. """

    def __init__(
        self,
        num_actions: int,
        num_hidden_units: int
    ):
        super().__init__()
        self.common = layers.Dense(num_hidden_units, activation = 'relu')
        # outputs scale, location params for mvn
        self.actor = ContinuousActor(num_actions, num_hidden_units) #layers.Dense(num_actions)
        self.critic = layers.Dense(1)

    def call(self, inputs: tf.Tensor):
        x = self.common(inputs)
        return self.actor(x), self.critic(x)

class sepActorCritic(tf.keras.Model):
    """combined actor-critic network. """

    def __init__(
        self,
        num_actions: int,
        num_hidden_units: int
    ):
        super().__init__()
        self.actor1 = layers.Dense(num_hidden_units, activation = 'relu')
        # outputs scale, location params for mvn
        self.actor2 = ContinuousActor(num_actions, num_hidden_units) 
        self.critic1 = layers.Dense(num_hidden_units, activation = 'relu')
        self.critic2 = layers.Dense(1)

    def call(self, inputs: tf.Tensor):
        a = self.actor1(inputs)
        c = self.critic1(inputs)
        return self.actor2(a), self.critic2(c)


In [9]:
class Agent():

    def __init__(
        self, 
        gamma: float, 
        lam: float,
        entropy_coef: float,
        vf_coef: float,
        scale: float,
        model: tf.keras.Model, 
        optimizer: tf.keras.optimizers.Optimizer,
        summary_writer,
        debug = False,
        normalise_rewards = False):

        # discount rate
        self.gamma = tf.constant(gamma)
        self.lam = tf.constant(lam)
        self.entropy_coef = tf.constant(entropy_coef)
        self.vf_coef = tf.constant(vf_coef)
        self.scale = scale
        self.model = model
        self.optimizer = optimizer

        # creates a dictionary of tensor arrays to write to
        self.memory = self._init_memory()
        self.loss = tf.keras.losses.Huber(reduction = tf.keras.losses.Reduction.NONE)

        self.model.compile(optimizer=self.optimizer)
        self.summary_writer = summary_writer

        # switch for analysis
        self.DEBUG = debug
        if self.DEBUG:
            self.debug_val = 0

        self.normalise_rewards = normalise_rewards

    def _init_memory(self):
        return {
                'action' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'state' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'next_state' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'action_probs': tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'values' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'rewards' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'done' : tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
        }

    
    def _get_action(self, state):

        policy, value = self.model(
                tf.expand_dims(tf.constant(state, tf.float32), 0)
            )
        action = policy.sample()[0]

        return self.scale * tf.tanh(action), policy, value
        # return tf.clip_by_value(action, -0.99, 0.99), policy, value
    
    def get_action(self, state, step):

        # run the model
        action, policy, _ = self._get_action(state)

        if self.DEBUG:
            with self.summary_writer.as_default():
                tf.summary.scalar('mean1', policy.mean()[0][0], step = self.debug_val)
                tf.summary.scalar('mean2', policy.mean()[0][1], step = self.debug_val)
                tf.summary.scalar('std1', policy.stddev()[0][0], step = self.debug_val)
                tf.summary.scalar('std2', policy.stddev()[0][1], step = self.debug_val)

            self.debug_val += 1

        return action

    def log(self, action, state, next_state, reward, done, step):

        """
        Logs results into memory - not all used necessarily
        """

        self.memory['action'].write(
            step, 
            tf.constant(action, tf.float32)
            ).mark_used()

        self.memory['state'].write(
            step, 
            tf.constant(state, tf.float32)
            ).mark_used()

        self.memory['next_state'].write(
            step, 
            tf.constant(next_state, tf.float32)
            ).mark_used()

        self.memory['rewards'].write(
            step, 
            tf.constant(reward, tf.float32)
            ).mark_used()

        self.memory['done'].write(
            step, 
            tf.constant(done, tf.int32)
            ).mark_used()

    def get_gae(self, rewards, values, dones, standardize):
        
        n = tf.shape(dones)[0] 
        advantages = tf.TensorArray(dtype=tf.float32, size=n)
        last_advantage = tf.constant(0.)

        for t in reversed(tf.range(tf.shape(rewards))):
            if t + 1 < tf.shape(rewards):
                
                delta = rewards[t] + self.gamma * values[t + 1] * (1 - dones[t + 1]) - values[t]

            else:
                
                delta = rewards[t] - values[t]

            
            advantage = delta + self.gamma * self.lam * (1 - dones[t]) * last_advantage
            advantages.write(t, advantage).mark_used()

        advantages = advantages.stack()[::-1]
        if standardize:
            advantages = ((advantages - tf.reduce_mean(advantages)) /
                            (tf.math.reduce_std(advantages) + eps))

        return advantages

    def get_expected_return(
        self,
        rewards: tf.Tensor,
        standardize: bool = True):
        """Compute expected returns"""

        ### could convert this to generalised advantage estimator
        ### get 'done' flags and multiply discounted sum and gamma by (1-done)
        ### also need value estimates
        ### do some checking, but if you're careful, the done value could enable you to do this process across batches
        ### means you could do multiple batches per update - this might help the algorithm learn
        n = tf.shape(rewards)[0] 
        returns = tf.TensorArray(dtype=tf.float32, size=n)

        # start at last reward and then accumulate reward sums into returns array
        rewards = rewards[::-1]
        discounted_sum = tf.constant(0.0)
        discounted_sum_shape = discounted_sum.shape
        for i in tf.range(n):
            reward = rewards[i]
            discounted_sum = reward + self.gamma * discounted_sum # discounted_sum= 0 for last reward (i.e. first element in loop)
            discounted_sum.set_shape(discounted_sum_shape)
            returns = returns.write(i, discounted_sum)
        returns = returns.stack()[::-1] # reverse order back to original

        if standardize:
            returns = ((returns - tf.reduce_mean(returns)) / 
                        (tf.math.reduce_std(returns) + eps))
        
        return returns 

    def update(self, episode: int):

        states = self.memory['state'].gather([i for i in tf.range(self.memory['state'].size())])
        # tf.expand_dims(self.memory['state'].gather([i for i in tf.range(self.memory['state'].size())]), 1)
        rewards = self.memory['rewards'].gather([i for i in tf.range(self.memory['rewards'].size())])
        actions = self.memory['action'].gather([i for i in tf.range(self.memory['action'].size())])
        dones = tf.cast(self.memory['done'].gather([i for i in tf.range(self.memory['done'].size())]), tf.float32)
        # returns = self.get_expected_return(rewards = rewards, standardize=self.normalise_rewards)

        with tf.GradientTape() as tape:

            policy, values = self.model(states)

            # adding values makes this into 'returns' and is optimised against the predicted value (i.e. output from model)
            # value to add should be saved from rollout
            advantages = self.get_gae(rewards, values, dones, self.normalise_rewards) #+ values

            # calculate the loss values
            loss = self.compute_loss(
                actions, 
                policy, 
                advantages,
                # values,
                # returns,
                episode)

        # compute the gradients from the loss
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        with self.summary_writer.as_default():
            tf.summary.scalar('loss', loss, step = episode)
            tf.summary.scalar('episode_reward', tf.reduce_sum(rewards), step = episode)

        # wipe memory for next episode
        self.memory = self._init_memory()
        
    def compute_loss(
        self,
        actions: tf.Tensor,
        policy: tf.Tensor,
        advantages: tf.Tensor,
        # values: tf.Tensor,
        # returns: tf.Tensor,
        episode: int
    ) -> tf.Tensor:
        """Computes combined actor-critic loss"""
        # print(f'returns: {tf.shape(returns)}; values: {tf.shape(values)}')
        # advantage = returns - tf.squeeze(values)

        critic_loss = tf.pow(advantages, 2, name='critic_loss')#self.loss(tf.squeeze(values), returns)#tf.math.square(advantage)
        # print(tf.reduce_mean(critic_loss))
        # print(tf.reduce_mean(advantages))

        entropy_loss = -policy.entropy()

        # back to original scale
        converted_actions = actions / self.scale
        # squashing correction
        log_probs = tf.expand_dims(policy.log_prob(converted_actions), axis = 1)
        # stopping gradient for advantage dramatically improves stability!
        actor_loss = -tf.reduce_sum(log_probs - tf.squeeze(tf.math.log(1 - tf.math.pow(converted_actions, 2) + 1.0e-10)), axis=1) * tf.stop_gradient(advantages)

        with self.summary_writer.as_default():
            tf.summary.scalar('actor_loss', tf.reduce_mean(actor_loss), step = episode)
            tf.summary.scalar('critic_loss', tf.reduce_mean(critic_loss), step = episode)
            tf.summary.scalar('entropy_loss', tf.reduce_mean(entropy_loss), step = episode)

        # critic_grad_check = tf.get_default_graph().get_operation_by_name('critic_loss')
        # print(get_gradient_function(critic_loss))
        return tf.reduce_mean(actor_loss + self.vf_coef*critic_loss + self.entropy_coef *entropy_loss)

def get_next_run(log_dir):
    next_run = max([0]+[int(j) for j in [i.split('_')[-1] for i in os.listdir(log_dir)] if j.isdigit()]) + 1
    return log_dir + f'/run_{next_run}'

def train(agent, env, num_episodes, seed):
    
    for i in range(num_episodes):
        state, _ = env.reset(seed = seed)
        done = False
        step = 0
        while not done:
            action = agent.get_action(state, step)
            next_state, reward, terminated, truncated, info = env.step(np.array(action))
            done = terminated or truncated
            agent.log(action, state, next_state, reward, done, step)
            state = next_state

            step += 1
        agent.update(i)

def test(agent, env, num_episodes, seed):

    for i in range(num_episodes):
        state, _ = env.reset(seed = seed)
        done = False
        step = 0
        while not done:
            action = agent.get_action(state, step)
            next_state, reward, terminated, truncated, info = env.step(np.array(action))
            done = terminated or truncated
            state = next_state

            step += 1
    env.close()


In [10]:
## create tensorboard logs
LOGS = './logs'
if not os.path.exists(LOGS):
    os.mkdir(LOGS)

In [11]:
ENV = 'LunarLanderContinuous-v2'#'Pendulum-v1'#

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
env = gym.make(ENV)
eps = np.finfo(np.float32).eps.item()

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0007, clipnorm=0.5)
num_actions = env.action_space.shape[0]
num_hidden_units = 256
# model = ActorCritic(num_actions, num_hidden_units)
model = sepActorCritic(num_actions, num_hidden_units)

log_dir = get_next_run(LOGS) 
print('Saving logs to: ', log_dir)
summary_writer = tf.summary.create_file_writer(logdir = log_dir)
agent = Agent(0.99,0.9, 0.0, 0.4, 1, model, optimizer, summary_writer, debug=False, normalise_rewards = True)

NUM_EPISODES=100
train(agent, env, NUM_EPISODES, seed)


Saving logs to:  ./logs/run_189


In [37]:
for i in range(1):
    state, _ = env.reset(seed = seed)
    done = False
    step = 0
    while not done:
        action = agent.get_action(state, step)
        next_state, reward, terminated, truncated, info = env.step(np.array(action))
        done = terminated or truncated
        agent.log(action, state, next_state, reward, done, step)
        state = next_state
        step +=1

In [40]:
rewards = agent.memory['rewards'].gather([i for i in tf.range(agent.memory['rewards'].size())])
states = agent.memory['state'].gather([i for i in tf.range(agent.memory['state'].size())])
dones = agent.memory['done'].gather([i for i in tf.range(agent.memory['done'].size())])

_, values = agent.model(states)

In [48]:
advantages = agent.get_gae(rewards, values, tf.cast(dones, tf.float32), True)

In [54]:
advantages**2

<tf.Tensor: shape=(164, 1), dtype=float32, numpy=
array([[1.49467209e+02],
       [6.89568281e+00],
       [1.84197411e-01],
       [5.71572363e-01],
       [2.62087435e-02],
       [6.23522811e-02],
       [4.74430025e-02],
       [1.76851332e-01],
       [5.11911884e-02],
       [4.70971828e-03],
       [9.79589224e-02],
       [2.93170698e-02],
       [6.84715062e-03],
       [3.07761040e-02],
       [2.10136399e-02],
       [1.26643879e-02],
       [7.88297712e-06],
       [1.16279072e-04],
       [1.08048451e-04],
       [2.68539414e-03],
       [1.25033213e-04],
       [1.50103704e-04],
       [1.15281509e-04],
       [8.81589111e-03],
       [7.94908870e-03],
       [2.95914686e-03],
       [5.48998360e-03],
       [4.22677733e-02],
       [7.22334860e-03],
       [1.62651972e-03],
       [4.95722564e-03],
       [3.92644026e-04],
       [1.34479925e-02],
       [6.74373889e-03],
       [1.74059942e-02],
       [2.38062363e-04],
       [4.24834026e-04],
       [2.39905179e-03],


In [199]:
agent.model.save_weights('./logs/run_154/model.keras')

In [24]:
from gymnasium.wrappers import NormalizeObservation
env = NormalizeObservation(env)
state, _ = env.reset()
state

array([-6.91900327e-05,  8.16159559e-03, -5.73932661e-03,  7.46810543e-04,
        8.02413463e-05,  1.56790813e-03,  0.00000000e+00,  0.00000000e+00])

In [36]:
NUM_EPISODES=3
env = gym.make(ENV, render_mode='human')
test(agent, env, NUM_EPISODES, seed)

## RNN / sequence model
Try to create an RL agent with an RNN network

In [46]:
class ContinuousActor(tf.keras.Model):
    """Actor that outputs a policy directly"""
    def __init__(
        self,
        num_actions: int,
        num_hidden_units: int
    ):
        super().__init__()
        self.means = layers.Dense(num_actions)
        self.stds = lambda x: tf.zeros(num_actions)
        # self.stds = layers.Dense(num_actions, activation='relu')
        # tf.ones(num_actions) * 0.25
    def call(self, inputs: tf.Tensor):
        means = self.means(inputs)
        stds = self.stds(inputs)
        # stds = tf.clip_by_value(stds, 1.0e-3, 1)
        return tfp.distributions.MultivariateNormalDiag(loc = means, scale_diag = tf.exp(stds))


class RecurrentActorCritic(tf.keras.Model):
    """combined actor-critic network. """

    def __init__(
        self,
        num_actions: int,
        num_hidden_units: int
    ):
        super().__init__()
        self.common = layers.LSTM(num_hidden_units)
        # outputs scale, location params for mvn
        self.actor = ContinuousActor(num_actions, num_hidden_units) 
        self.critic = layers.Dense(1)

    def call(self, inputs: tf.Tensor):
        x = self.common(inputs)
        return self.actor(x), self.critic(x)




In [48]:
class RecurrentAgent():

    def __init__(
        self, 
        gamma: float, 
        entropy_coef: float,
        vf_coef: float,
        window_size: int,
        scale: float,
        model: tf.keras.Model, 
        optimizer: tf.keras.optimizers.Optimizer,
        summary_writer,
        debug = False,
        normalise_rewards = False):
        
        # discount rate
        self.gamma = tf.constant(gamma)
        self.entropy_coef = tf.constant(entropy_coef)
        self.vf_coef = tf.constant(vf_coef)
        self.window_size = tf.constant(window_size)
        self.scale = tf.constant(scale)
        self.model = model
        self.optimizer = optimizer

        # creates a dictionary of tensor arrays to write to
        self.memory = self._init_memory()
        self.loss = tf.keras.losses.Huber(reduction = tf.keras.losses.Reduction.NONE)

        self.model.compile(optimizer=self.optimizer)
        self.summary_writer = summary_writer

        # switch for analysis
        self.DEBUG = debug
        if self.DEBUG:
            self.debug_val = 0

        self.normalise_rewards = normalise_rewards

    def _init_memory(self):
        return {
                'action' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'state' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'next_state' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'action_probs': tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'values' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'rewards' : tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True),
                'done' : tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
        }

    def _pad_input(self, new_state):
        
        # get previous states and combine with new_state
        if self.memory['state'].size() > 0:
            last_n = self.memory['state'].size()
            first_n = np.max([0, last_n - self.window_size], 0)
            prev_states = self.memory['state'].gather([i for i in tf.range(first_n, last_n)])
            combined_states = tf.concat([tf.expand_dims(new_state, 0), prev_states], axis =0)
        else:
            combined_states = tf.expand_dims(new_state, 0)

        # get size of padding - 0 if we have enough
        pad_size = np.max([0, self.window_size-tf.shape(combined_states)[0]], axis = 0)
        
        # get the padded vals
        padded = tf.pad(combined_states, [[0,pad_size],[0,0]])
        padded = tf.expand_dims(padded, 0)

        return padded
    
    def _get_action(self, state):

        padded = self._pad_input(state)

        policy, value = self.model(padded)
        action = policy.sample()[0]

        return self.scale * tf.tanh(action), policy, value

    
    def get_action(self, state, step):

        # run the model
        action, policy, _ = self._get_action(state)

        if self.DEBUG:
            with self.summary_writer.as_default():
                tf.summary.scalar('mean1', policy.mean()[0][0], step = self.debug_val)
                tf.summary.scalar('mean2', policy.mean()[0][1], step = self.debug_val)
                tf.summary.scalar('std1', policy.stddev()[0][0], step = self.debug_val)
                tf.summary.scalar('std2', policy.stddev()[0][1], step = self.debug_val)

            self.debug_val += 1

        return action

    def log(self, action, state, next_state, reward, done, step):

        """
        Logs results into memory - not all used necessarily
        """

        self.memory['action'].write(
            step, 
            tf.constant(action, tf.float32)
            ).mark_used()

        self.memory['state'].write(
            step, 
            tf.constant(state, tf.float32)
            ).mark_used()

        self.memory['next_state'].write(
            step, 
            tf.constant(next_state, tf.float32)
            ).mark_used()

        self.memory['rewards'].write(
            step, 
            tf.constant(reward, tf.float32)
            ).mark_used()

        self.memory['done'].write(
            step, 
            tf.constant(done, tf.int32)
            ).mark_used()

    def get_expected_return(
        self,
        rewards: tf.Tensor,
        standardize: bool = True):
        """Compute expected returns"""

        n = tf.shape(rewards)[0] 
        returns = tf.TensorArray(dtype=tf.float32, size=n)

        # start at last reward and then accumulate reward sums into returns array
        rewards = rewards[::-1]
        discounted_sum = tf.constant(0.0)
        discounted_sum_shape = discounted_sum.shape
        for i in tf.range(n):
            reward = rewards[i]
            discounted_sum = reward + self.gamma * discounted_sum # discounted_sum= 0 for last reward (i.e. first element in loop)
            discounted_sum.set_shape(discounted_sum_shape)
            returns = returns.write(i, discounted_sum)
        returns = returns.stack()[::-1] # reverse order back to original

        if standardize:
            returns = ((returns - tf.reduce_mean(returns)) / 
                        (tf.math.reduce_std(returns) + eps))
        
        return returns

    def _prepare_states_for_lstm(self, states):
        """wraps states into lstm sequence format"""
        ## need the initial k 

        initial_padded_states = tf.convert_to_tensor([tf.pad(states[:(i+1)][::-1], [[0,np.max([0, self.window_size-(i+1)], axis = 0)],[0,0]]) for i in tf.range(0, self.window_size)])
        remaining_states = tf.convert_to_tensor([states[(i-self.window_size):i] for i in tf.range(self.window_size, tf.shape(states)[0])])
        combined_states = tf.concat([initial_padded_states, remaining_states], axis = 0)

        return combined_states
        

    def update(self, episode: int):

        states = self.memory['state'].gather([i for i in tf.range(self.memory['state'].size())])
        states = self._prepare_states_for_lstm(states)
        rewards = self.memory['rewards'].gather([i for i in tf.range(self.memory['rewards'].size())])
        actions = self.memory['action'].gather([i for i in tf.range(self.memory['action'].size())])
        returns = self.get_expected_return(rewards = rewards, standardize=self.normalise_rewards)

        with tf.GradientTape() as tape:

            policy, values = self.model(states)

            # calculate the loss values
            loss = self.compute_loss(
                actions, 
                policy, 
                values,
                returns,
                episode)

        # compute the gradients from the loss
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        with self.summary_writer.as_default():
            tf.summary.scalar('loss', loss, step = episode)
            tf.summary.scalar('episode_reward', tf.reduce_sum(rewards), step = episode)

        # wipe memory for next episode
        self.memory = self._init_memory()
        
    def compute_loss(
        self,
        actions: tf.Tensor,
        policy: tf.Tensor,
        values: tf.Tensor,
        returns: tf.Tensor,
        episode: int
    ) -> tf.Tensor:
        """Computes combined actor-critic loss"""
        # print(f'returns: {tf.shape(returns)}; values: {tf.shape(values)}')
        advantage = returns - tf.squeeze(values)

        critic_loss = self.vf_coef*self.loss(tf.squeeze(values), returns)#tf.math.square(advantage)

        entropy_loss = -self.entropy_coef * policy.entropy()

        converted_actions = actions/self.scale
        # squashing correction
        log_probs = tf.expand_dims(policy.log_prob(converted_actions), axis = 1)
        # stopping gradient for advantage dramatically improves stability!
        actor_loss = tf.reduce_sum(-log_probs - tf.squeeze(tf.math.log(1 - tf.math.pow(converted_actions, 2) + 1.0e-10)), axis=1) * tf.stop_gradient(advantage)

        with self.summary_writer.as_default():
            tf.summary.scalar('actor_loss', tf.reduce_mean(actor_loss), step = episode)
            tf.summary.scalar('critic_loss', tf.reduce_mean(critic_loss), step = episode)
            tf.summary.scalar('entropy_loss', tf.reduce_mean(entropy_loss), step = episode)

        return tf.reduce_mean(actor_loss + critic_loss + entropy_loss)

In [50]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
env = gym.make('LunarLanderContinuous-v2')
eps = np.finfo(np.float32).eps.item()

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0007, clipnorm=0.5)
num_actions = env.action_space.shape[0]
num_hidden_units = 256
model = RecurrentActorCritic(num_actions, num_hidden_units)

log_dir = get_next_run(LOGS) 
print('Saving logs to: ', log_dir)
summary_writer = tf.summary.create_file_writer(logdir = log_dir)
agent2 = RecurrentAgent(0.99, 0.0, 0.4, 8, 1., model, optimizer, summary_writer, debug=True, normalise_rewards=False)

NUM_EPISODES=100
train(agent2, env, NUM_EPISODES, seed)

Saving logs to:  ./logs/run_165
